In [67]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from pandas_profiling import ProfileReport
import re 
import warnings
warnings.filterwarnings("ignore")

In [68]:
raw_dataset = pd.read_json ('Data/imdb_dataset.json')
raw_dataset

,title,rating,year,users_rating,votes,metascore,img_url,countries,languages,actors,genre,tagline,description,directors,runtime,imdb_url
0,12 Strong,R,2018.0,6.6,"42,919",54.0,https://m.media-amazon.com/images/M/MV5BNTEzMj...,[USA],"[English, Dari, Russian, Spanish, Uzbek]","[Chris Hemsworth, Michael Shannon, Michael Peñ...","[Action, Drama, History, War]",The Declassified True Story of the Horse Soldiers,12 Strong tells the story of the first Special...,[Nicolai Fuglsig],130 min,https://www.imdb.com/title/tt1413492/
1,The Silence of the Lambs,R,1991.0,8.6,"1,091,846",85.0,https://m.media-amazon.com/images/M/MV5BNjNhZT...,[USA],[English],"[Jodie Foster, Lawrence A. Bonney, Kasi Lemmon...","[Crime, Drama, Thriller]",Dr. Hannibal Lecter. Brilliant. Cunning. Psych...,A young FBI cadet must receive the help of an ...,[Jonathan Demme],118 min,https://www.imdb.com/title/tt0102926/
2,Blade Runner,R,1982.0,8.2,"609,925",89.0,https://m.media-amazon.com/images/M/MV5BNzQzMz...,"[USA, Hong Kong]","[English, German, Cantonese, Japanese, Hungari...","[Harrison Ford, Rutger Hauer, Sean Young, Edwa...","[Sci-Fi, Thriller]",A Futuristic Vision Perfected [2007 Final Cut],A blade runner must pursue and terminate four ...,[Ridley Scott],117 min,https://www.imdb.com/title/tt0083658/
3,Logan,R,2017.0,8.1,"525,404",77.0,https://m.media-amazon.com/images/M/MV5BYzc5MT...,[USA],"[English, Spanish]","[Hugh Jackman, Patrick Stewart, Dafne Keen, Bo...","[Action, Drama, Sci-Fi, Thriller]",One Last Time.,"In the near future, a weary Logan cares for an...",[James Mangold],137 min,https://www.imdb.com/title/tt3315342/
4,Harry Potter and the Chamber of Secrets,PG,2002.0,7.4,"475,434",63.0,https://m.media-amazon.com/images/M/MV5BMTcxOD...,"[UK, USA, Germany]",[English],"[Daniel Radcliffe, Rupert Grint, Emma Watson, ...","[Adventure, Family, Fantasy, Mystery]",The second year begins November 15,An ancient prophecy seems to be coming true wh...,[Chris Columbus],161 min,https://www.imdb.com/title/tt0295297/
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58618,Black Dragon,None,2003.0,5.7,15,NaN,https://m.media-amazon.com/images/M/MV5BMTUwOT...,[USA],[English],"[Stephen Benitzhar, Lewis Borders, Aliina Buck...","[Action, Fantasy, Thriller]",Would you exchange your soul to live your fant...,One man's life is altered by the hand of fate....,[Charley Rice],None,https://www.imdb.com/title/tt0770170/
58619,Hold That Line,Approved,1952.0,6.7,199,NaN,https://m.media-amazon.com/images/M/MV5BYzRiYj...,[USA],[English],"[Leo Gorcey, Huntz Hall, John Bromfield, Veda ...","[Action, Comedy, Sport]",They're a SCREAM Team Of All-American RIOTS!,"In a precursor to Trading Places (1983), the B...",[William Beaudine],67 min,https://www.imdb.com/title/tt0044714/
58620,Jam Session,Approved,1944.0,6.4,132,NaN,https://m.media-amazon.com/images/M/MV5BMjk1Zj...,[USA],[English],"[Ann Miller, Jess Barker, Charles D. Brown, Ed...","[Comedy, Music]",IT'S A JIVE HIT! (original print ad-all caps),Terry Baxter is a never-say-die Hollywood newc...,[Charles Barton],77 min,https://www.imdb.com/title/tt0036966/
58621,The Black Pirates,None,1954.0,5.5,31,NaN,https://m.media-amazon.com/images/M/MV5BZDQyOG...,"[USA, Mexico, El Salvador]",[English],"[Anthony Dexter, Martha Roth, Lon Chaney Jr., ...",[Adventure],Wild raiders of the tropic seas!,Pirates searching for treasure take over a sma...,[Allen H. Miner],74 min,https://www.imdb.com/title/tt0046787/


In [171]:
#Remove movies with less than 5 imdb rating and langauges other than english and hindi

#First replace Empty values "" with 0
raw_dataset['users_rating']=raw_dataset['users_rating'].replace("",0)

#movies with rating more than 5 will be included in the list
movie_rating = raw_dataset[raw_dataset['users_rating']>5]

#Only released movies to be included in the list
#movie_rel = movie_more5[movie_more5['status']=='Released']

#Convert the title to lowercase
movie_rating['title'] = movie_rating['title'].str.lower()

#Select the major languages
#movie_lang = movie_rel[movie_rel['original_language'].isin(['en','ja','it','de','es','ru','ko','hi'])== True]

#Select movies with more than 500 vote counts
#movie_vote = movie_rating[movie_rating['votes']>10000]

#Reset index after removal of rows
movie_data = movie_rating.reset_index().drop(columns='index',axis=1)

In [172]:
movies = movie_data[['title','countries','genre','languages','actors','tagline','description','directors','users_rating','votes']].replace(np.nan, '', regex=True)
movie_dataset = movies[['title','countries','genre','languages','actors','tagline','description','directors']].astype(str)
movie_dataset

,title,countries,genre,languages,actors,tagline,description,directors
0,12 strong,['USA'],"['Action', 'Drama', 'History', 'War']","['English', 'Dari', 'Russian', 'Spanish', 'Uzb...","['Chris Hemsworth', 'Michael Shannon', 'Michae...",The Declassified True Story of the Horse Soldiers,12 Strong tells the story of the first Special...,['Nicolai Fuglsig']
1,the silence of the lambs,['USA'],"['Crime', 'Drama', 'Thriller']",['English'],"['Jodie Foster', 'Lawrence A. Bonney', 'Kasi L...",Dr. Hannibal Lecter. Brilliant. Cunning. Psych...,A young FBI cadet must receive the help of an ...,['Jonathan Demme']
2,blade runner,"['USA', 'Hong Kong']","['Sci-Fi', 'Thriller']","['English', 'German', 'Cantonese', 'Japanese',...","['Harrison Ford', 'Rutger Hauer', 'Sean Young'...",A Futuristic Vision Perfected [2007 Final Cut],A blade runner must pursue and terminate four ...,['Ridley Scott']
3,logan,['USA'],"['Action', 'Drama', 'Sci-Fi', 'Thriller']","['English', 'Spanish']","['Hugh Jackman', 'Patrick Stewart', 'Dafne Kee...",One Last Time.,"In the near future, a weary Logan cares for an...",['James Mangold']
4,harry potter and the chamber of secrets,"['UK', 'USA', 'Germany']","['Adventure', 'Family', 'Fantasy', 'Mystery']",['English'],"['Daniel Radcliffe', 'Rupert Grint', 'Emma Wat...",The second year begins November 15,An ancient prophecy seems to be coming true wh...,['Chris Columbus']
...,...,...,...,...,...,...,...,...
44777,tiburon,['USA'],"['Drama', 'Thriller']",['English'],"['Christopher Behrens', 'Denise Behrens', 'Sco...",Reflections hide what stirs beneath the surface,Two brothers battle each other for control of ...,['Christopher Behrens']
44778,black dragon,['USA'],"['Action', 'Fantasy', 'Thriller']",['English'],"['Stephen Benitzhar', 'Lewis Borders', 'Aliina...",Would you exchange your soul to live your fant...,One man's life is altered by the hand of fate....,['Charley Rice']
44779,hold that line,['USA'],"['Action', 'Comedy', 'Sport']",['English'],"['Leo Gorcey', 'Huntz Hall', 'John Bromfield',...",They're a SCREAM Team Of All-American RIOTS!,"In a precursor to Trading Places (1983), the B...",['William Beaudine']
44780,jam session,['USA'],"['Comedy', 'Music']",['English'],"['Ann Miller', 'Jess Barker', 'Charles D. Brow...",IT'S A JIVE HIT! (original print ad-all caps),Terry Baxter is a never-say-die Hollywood newc...,['Charles Barton']


In [173]:
def combine_features(row):
    try:
        return row['title'] +30*(" "+row["countries"])+30*(" "+row["languages"])+35*(" "+row["actors"])+35*(" "+row["genre"])+35*(" "+row["directors"])+" "+row["description"]+" "+row["tagline"]
    except:
        print("Error:",row)

In [174]:
#Combine title, genre, overview and production companies
#Thus a user will be recommended movies that are similar in genre, overiew and production company

movie_dataset["combined_features"] = movie_dataset.apply(combine_features,axis=1)

#Convert the string to lower case
movie_dataset["combined_features"]= movie_dataset["combined_features"].str.lower()

#Lets view one of the combine feature
movie_dataset["combined_features"][0]

'12 strong [\'usa\'] [\'usa\'] [\'usa\'] [\'usa\'] [\'usa\'] [\'usa\'] [\'usa\'] [\'usa\'] [\'usa\'] [\'usa\'] [\'usa\'] [\'usa\'] [\'usa\'] [\'usa\'] [\'usa\'] [\'usa\'] [\'usa\'] [\'usa\'] [\'usa\'] [\'usa\'] [\'usa\'] [\'usa\'] [\'usa\'] [\'usa\'] [\'usa\'] [\'usa\'] [\'usa\'] [\'usa\'] [\'usa\'] [\'usa\'] [\'english\', \'dari\', \'russian\', \'spanish\', \'uzbek\'] [\'english\', \'dari\', \'russian\', \'spanish\', \'uzbek\'] [\'english\', \'dari\', \'russian\', \'spanish\', \'uzbek\'] [\'english\', \'dari\', \'russian\', \'spanish\', \'uzbek\'] [\'english\', \'dari\', \'russian\', \'spanish\', \'uzbek\'] [\'english\', \'dari\', \'russian\', \'spanish\', \'uzbek\'] [\'english\', \'dari\', \'russian\', \'spanish\', \'uzbek\'] [\'english\', \'dari\', \'russian\', \'spanish\', \'uzbek\'] [\'english\', \'dari\', \'russian\', \'spanish\', \'uzbek\'] [\'english\', \'dari\', \'russian\', \'spanish\', \'uzbek\'] [\'english\', \'dari\', \'russian\', \'spanish\', \'uzbek\'] [\'english\', \'da

In [175]:
#Lets remove the unwanted parentheses, id, name, punctuation

movie_dataset.astype(str)['combined_features'].map(lambda x:  type(x))

replace_val = ["{'id'","'name':","'},","[","]","'",",","}]","{","}",".",":","'",'[0-9]','id','--','"']

for i in replace_val:
  movie_dataset["combined_features"] = movie_dataset["combined_features"].str.replace(i, '')

for i in range(len(movie_dataset)):
    movie_dataset["combined_features"][i] = re.sub(' +', ' ', movie_dataset["combined_features"][i]) 

movie_dataset["combined_features"][0]

' strong usa usa usa usa usa usa usa usa usa usa usa usa usa usa usa usa usa usa usa usa usa usa usa usa usa usa usa usa usa usa english dari russian spanish uzbek english dari russian spanish uzbek english dari russian spanish uzbek english dari russian spanish uzbek english dari russian spanish uzbek english dari russian spanish uzbek english dari russian spanish uzbek english dari russian spanish uzbek english dari russian spanish uzbek english dari russian spanish uzbek english dari russian spanish uzbek english dari russian spanish uzbek english dari russian spanish uzbek english dari russian spanish uzbek english dari russian spanish uzbek english dari russian spanish uzbek english dari russian spanish uzbek english dari russian spanish uzbek english dari russian spanish uzbek english dari russian spanish uzbek english dari russian spanish uzbek english dari russian spanish uzbek english dari russian spanish uzbek english dari russian spanish uzbek english dari russian spanish uz

In [176]:
#Copy movie dataset to df
df= movie_dataset.copy(deep=True)

#Create a column named index
df['index'] = df.index

In [177]:
df

,title,countries,genre,languages,actors,tagline,description,directors,combined_features,index
0,12 strong,['USA'],"['Action', 'Drama', 'History', 'War']","['English', 'Dari', 'Russian', 'Spanish', 'Uzb...","['Chris Hemsworth', 'Michael Shannon', 'Michae...",The Declassified True Story of the Horse Soldiers,12 Strong tells the story of the first Special...,['Nicolai Fuglsig'],strong usa usa usa usa usa usa usa usa usa us...,0
1,the silence of the lambs,['USA'],"['Crime', 'Drama', 'Thriller']",['English'],"['Jodie Foster', 'Lawrence A. Bonney', 'Kasi L...",Dr. Hannibal Lecter. Brilliant. Cunning. Psych...,A young FBI cadet must receive the help of an ...,['Jonathan Demme'],the silence of the lambs usa usa usa usa usa u...,1
2,blade runner,"['USA', 'Hong Kong']","['Sci-Fi', 'Thriller']","['English', 'German', 'Cantonese', 'Japanese',...","['Harrison Ford', 'Rutger Hauer', 'Sean Young'...",A Futuristic Vision Perfected [2007 Final Cut],A blade runner must pursue and terminate four ...,['Ridley Scott'],blade runner usa hong kong usa hong kong usa h...,2
3,logan,['USA'],"['Action', 'Drama', 'Sci-Fi', 'Thriller']","['English', 'Spanish']","['Hugh Jackman', 'Patrick Stewart', 'Dafne Kee...",One Last Time.,"In the near future, a weary Logan cares for an...",['James Mangold'],logan usa usa usa usa usa usa usa usa usa usa ...,3
4,harry potter and the chamber of secrets,"['UK', 'USA', 'Germany']","['Adventure', 'Family', 'Fantasy', 'Mystery']",['English'],"['Daniel Radcliffe', 'Rupert Grint', 'Emma Wat...",The second year begins November 15,An ancient prophecy seems to be coming true wh...,['Chris Columbus'],harry potter and the chamber of secrets uk usa...,4
...,...,...,...,...,...,...,...,...,...,...
44777,tiburon,['USA'],"['Drama', 'Thriller']",['English'],"['Christopher Behrens', 'Denise Behrens', 'Sco...",Reflections hide what stirs beneath the surface,Two brothers battle each other for control of ...,['Christopher Behrens'],tiburon usa usa usa usa usa usa usa usa usa us...,44777
44778,black dragon,['USA'],"['Action', 'Fantasy', 'Thriller']",['English'],"['Stephen Benitzhar', 'Lewis Borders', 'Aliina...",Would you exchange your soul to live your fant...,One man's life is altered by the hand of fate....,['Charley Rice'],black dragon usa usa usa usa usa usa usa usa u...,44778
44779,hold that line,['USA'],"['Action', 'Comedy', 'Sport']",['English'],"['Leo Gorcey', 'Huntz Hall', 'John Bromfield',...",They're a SCREAM Team Of All-American RIOTS!,"In a precursor to Trading Places (1983), the B...",['William Beaudine'],hold that line usa usa usa usa usa usa usa usa...,44779
44780,jam session,['USA'],"['Comedy', 'Music']",['English'],"['Ann Miller', 'Jess Barker', 'Charles D. Brow...",IT'S A JIVE HIT! (original print ad-all caps),Terry Baxter is a never-say-die Hollywood newc...,['Charles Barton'],jam session usa usa usa usa usa usa usa usa us...,44780


In [178]:
#Create count matrix from this new combined column

cv = CountVectorizer()

count_matrix = cv.fit_transform(df["combined_features"])

#Compute the Cosine Similarity based on the count_matrix

cosine_sim = cosine_similarity(count_matrix) 
movie_user_likes = "death and the maiden"

In [179]:
#Helper functions to obtain index of movie from its title

def get_title_from_index(index):
    return df[df.index == index]["title"].values[0]

def get_index_from_title(title):
    return df[df.title == title]["index"].values[0]

In [192]:
#Get index of this movie from its title

movie_index = df[df.title == movie_user_likes]["index"].values[0]

similar_movies =  list(enumerate(cosine_sim[movie_index]))


#Get a list of similar movies in descending order of similarity score

sorted_similar_movies = sorted(similar_movies,key=lambda x:x[1],reverse=True)

# Print titles of first 5 movies
i=0
for element in sorted_similar_movies:
    if df[df.index == element[0]]["title"].values[0]==movie_user_likes:
        pass
    else:
        print(df[df.index == element[0]]["title"].values[0],round(element[1],2))
    i=i+1
    if i>5:
        break

con men 0.29
these words must be on your heart 0.27
the secrets of my heart 0.27
revenge of the fists of fury 0.27
abortion! 0.27


In [194]:
df[df.index == element[0]]["title"].values[0]

'abortion!'

In [255]:
def results(movie_name):
    movie_name = movie_name.lower()
    movie_data = pd.read_csv('Appdata/Movie_dataset.csv')
    movie_data = movie_data.drop(['Unnamed: 0','genres','overview','production_companies'],axis=1)
    #movie_data = movie_data.iloc[:100]

    if movie_name not in movie_data['title'].unique():
        return 'Movie not in Database'
    else:
        cv = CountVectorizer(stop_words='english')
        count_matrix = cv.fit_transform(movie_data['combined_features'])
        cosine_sim = cosine_similarity(count_matrix)

        movie_index = movie_data[movie_data.title == movie_name]['index'].values[0]

        similar_movies = list(enumerate(cosine_sim[movie_index]))
        sorted_similar_movies = sorted(similar_movies,key=lambda x:x[1],reverse=True)

        result=[]
        score=[]
        i=0
        for element in sorted_similar_movies:
            if movie_data[movie_data.index == element[0]]["title"].values[0] == movie_name:
                pass
            else:
                result.append(movie_data[movie_data.index == element[0]]["title"].values[0])
                score.append(round(element[1],2))
                i=i+1
                if i>9:
                    break
                    
        zip_iterator = zip(result, score)


        res_dct = dict(zip_iterator)


        return res_dct

In [256]:
results("Toy Story")

{'toy story 2': 0.62,
 'toy story 3': 0.6,
 'toy story of terror!': 0.45,
 'turbo': 0.44,
 'over the hedge': 0.44,
 'the boss baby': 0.44,
 'piper': 0.43,
 'finding dory': 0.43,
 'despicable me 2': 0.43,
 'penguins of madagascar': 0.41}

In [252]:
res_dct={}


for i in range(len(ans[0])):
    print(ans[0][i])
    print(ans[1][i])
    
res_dct

toy story 2
0.62
toy story 3
0.6
toy story of terror!
0.45
turbo
0.44
over the hedge
0.44
the boss baby
0.44
piper
0.43
finding dory
0.43
despicable me 2
0.43
penguins of madagascar
0.41


{}

In [246]:
res_dct= dict()

res_dct["Jheel"].append("Guru")
res_dct

{}

'toy story 2'

In [237]:
my_dict = {"Name":[],"Address":[],"Age":[]};

my_dict["Name"].append("Guru")
my_dict["Address"].append("Mumbai")
my_dict["Age"].append(30)
print(my_dict)

{'Name': ['Guru'], 'Address': ['Mumbai'], 'Age': [30]}


In [1]:
from imdb import IMDb

# create an instance of the IMDb class
ia = IMDb()

# get a movie
movie = ia.get_movie('0133093')

# print the names of the directors of the movie
print('Directors:')
for director in movie['directors']:
    print(director['name'])

# print the genres of the movie
print('Genres:')
for genre in movie['genres']:
    print(genre)

# search for a person name
people = ia.search_person('Mel Gibson')
for person in people:
   print(person.personID, person['name'])

Directors:
Lana Wachowski
Lilly Wachowski
Genres:
Action
Sci-Fi
0000154 Mel Gibson
2153167 Mel Gibson
0317112 Mel Gibson
3323885 Adam Taylor
2800055 Mel Gibson
3742421 Melina Mari Bryant   nickname
0316945 Donal Gibson
0317195 Terel Gibson
4047961 Noel Gibson
0316878 Cal Gibson
2106809 Raquel Gibson
3912400 Jamel Gibson
0316950 E.L. Gibson
2798174 Daniel Gibson
1812763 Rachel Gibson
7454542 L. Gibson
9235742 Daniel Gibson
2988370 Mal Gibson
0316915 Daniel Gibson
1279448 Mabel Gibson
